<a href="https://colab.research.google.com/github/LukasPurbaW/100_Days_of_Deep_Learning/blob/main/Day_5_ANN_for_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : https://drive.google.com/file/d/1rxj60Mm2_gWcDqNldS3x38Ue8opQcuR6/view?usp=sharing

In [2]:
# Now we will learn about using ANN for semi-real scenario with business question to create segmentation model which customers have a really high possibility of churn rates
# This could be applied in a million of scenarios such as credit approval, insurance, promo approval, voucher, or another things.

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

sheet_url = 'https://drive.google.com/file/d/1rxj60Mm2_gWcDqNldS3x38Ue8opQcuR6/view?usp=sharing'
sheet_url ='https://drive.google.com/uc?id=' + sheet_url.split('/')[-2]
Bank_Churn = pd.read_csv(sheet_url,encoding= 'unicode_escape')
Bank_Churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Create the dataset

In [7]:
print(Bank_Churn.isna().sum())
# No null values, nice

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


In [8]:
X = Bank_Churn.iloc[:, 3:-1].values
y = Bank_Churn.iloc[:, -1].values

## Encoding the geograohy and gender

In [9]:
# For gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [10]:
# For Geography
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

## Split Dataset

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Build and Train ANN

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=36, activation='relu'))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size = 8, epochs = 100)

Epoch 1/100
1000/1000 [==============================] - 3s 2ms/step - loss: 0.4343 - accuracy: 0.8115
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3649 - accuracy: 0.8519
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3487 - accuracy: 0.8576
Epoch 4/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3426 - accuracy: 0.8601
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3372 - accuracy: 0.8636
Epoch 6/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3347 - accuracy: 0.8640
Epoch 7/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3323 - accuracy: 0.8639
Epoch 8/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3282 - accuracy: 0.8658
Epoch 9/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3265 - accuracy: 0.8696
Epoch 10/100
1000/1000 [==============================] - 2s 2ms/step - l

## Predicting Test Result

In [23]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [24]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1454  141]
 [ 175  230]]


0.842

In [26]:
print(model.predict(sc.transform([[1, 0, 2, 600, 1, 40, 3, 50000, 2, 1, 1, 50000]])) > 0.5)

[[False]]
